In [8]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from scipy import stats

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import accuracy_score, precision_score, recall_score
import random

from sklearn.utils import resample

import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import imdb
import tensorflow as tf

In [ ]:
class UNet(nn.Module):
    def __init__(self, embedding_size=64):
        super(UNet, self).__init__()

        # Define the encoder part
        self.enc_conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.enc_conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

        # Define the bottleneck part
        self.bottleneck_conv = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(7*7*64, embedding_size)

    def forward(self, x):
        # Encoder
        x = F.relu(self.enc_conv1(x))
        x = self.pool(x)
        x = F.relu(self.enc_conv2(x))
        x = self.pool(x)

        # Bottleneck
        x = F.relu(self.bottleneck_conv(x))
        x = self.flatten(x)
        x = self.fc(x)

        return x

model = UNet()

In [9]:
def imbalance_classes(X, y, imb_type='exp', imb_factor=0.01):
    np.random.seed(42)
    unique_classes = np.unique(y)
    num_classes = len(unique_classes)
    img_max = len(X) / num_classes

    img_num_per_cls = []
    if imb_type == 'exp':
        for cls_idx in range(num_classes):
            num = img_max * (imb_factor**(cls_idx / (num_classes - 1.0)))
            img_num_per_cls.append(int(num))
    elif imb_type == 'step':
        for cls_idx in range(num_classes // 2):
            img_num_per_cls.append(int(img_max))
        for cls_idx in range(num_classes // 2):
            img_num_per_cls.append(int(img_max * imb_factor))
    else:
        img_num_per_cls.extend([int(img_max)] * num_classes)

    X_imb, y_imb = [], []
    for cls, img_num in zip(unique_classes, img_num_per_cls):
        idx = np.random.choice(np.where(y == cls)[0], img_num, replace=False)
        X_imb.append(X[idx])
        y_imb.append(y[idx])

    X_imb = np.concatenate(X_imb)
    y_imb = np.concatenate(y_imb)

    return X_imb, y_imb

In [10]:
def load_imbalanced_cifar10(imbalance_type='exp', imbalance_ratio=0.01):
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    X_train_imb, y_train_imb = imbalance_classes(X_train, y_train.flatten(), imbalance_type, imbalance_ratio)
    return X_train_imb, X_test, y_train_imb, y_test.flatten()

In [ ]:
X_train_imb, X_test, y_train_imb, y_test = load_imbalanced_cifar10(imbalance_type='exp', imbalance_ratio=0.1)

170498071/170498071 [==============================] - 2s 0us/step


In [11]:
class UNet(nn.Module):
    def __init__(self, embedding_size=64):
        super(UNet, self).__init__()

        # Modified to accept 3-channel input
        self.enc_conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.enc_conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

        # Define the bottleneck part
        self.bottleneck_conv = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(64 * 8 * 8, embedding_size)  # Adjusted for CIFAR10 size after pooling

    def forward(self, x):
        # Encoder
        x = F.relu(self.enc_conv1(x))
        x = self.pool(x)
        x = F.relu(self.enc_conv2(x))
        x = self.pool(x)

        # Bottleneck
        x = F.relu(self.bottleneck_conv(x))
        x = self.flatten(x)
        x = self.fc(x)

        return x

In [12]:
X_train_tensor = torch.from_numpy(X_train_imb).float()
X_train_tensor = X_train_tensor.permute(0, 3, 1, 2)

In [13]:
model = UNet()

In [15]:
model(X_train_tensor)

tensor([[-2.8296, 10.3660,  1.0588,  ..., -4.9630, -4.3389,  7.5269],
        [-1.7559,  5.4672,  1.0590,  ..., -4.1336, -5.3350,  6.0308],
        [-0.9531, 10.3312,  3.0008,  ..., -5.3253, -5.1206,  7.5419],
        ...,
        [-3.2185,  7.0926,  4.4474,  ..., -5.7924, -4.7102,  9.2088],
        [-2.5101,  5.4834,  1.9924,  ..., -0.9995, -4.1074,  7.3890],
        [-6.0206,  8.9397,  6.8829,  ..., -2.8011, -3.8728,  6.5218]],
       grad_fn=<AddmmBackward0>)